In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor

In [ ]:
path = '/content/drive/MyDrive/is665_004/Final/'

In [ ]:
train_df = pd.read_csv(path + 'data/train.csv')
test_df = pd.read_csv(path + 'data/test.csv')

In [ ]:
train_df

,ID,Age,Gender,Education_Status,Employment_Status,Working_Week (Yearly),Industry_Status,Occupation_Status,Race,Hispanic_Origin,...,Citizenship,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Tax_Status,Gains,Losses,Dividends,Income_Status,Income
0,TRAIN_00000,63,M,Middle (7-8),Full-Time,4,Social Services,Services,White,All other,...,Native,US,US,US,Nonfiler,0,0,0,Unknown,425
1,TRAIN_00001,37,M,Associates degree (Vocational),Full-Time,52,Entertainment,Services,White,All other,...,Native,US,US,US,Single,0,0,0,Under Median,0
2,TRAIN_00002,58,F,High graduate,Full-Time,52,Manufacturing (Non-durable),Admin Support (include Clerical),Black,All other,...,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),3411,0,0,Under Median,860
3,TRAIN_00003,44,M,High graduate,Full-Time,52,Retail,Technicians & Support,White,All other,...,Native,US,US,US,Single,0,0,0,Under Median,850
4,TRAIN_00004,37,F,High graduate,Full-Time,52,Retail,Sales,White,All other,...,Native,US,US,US,Head of Household (HOH),0,0,0,Unknown,570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,TRAIN_19995,33,M,High graduate,Children or Armed Forces,52,Manufacturing (Durable),Handlers/Cleaners,White,All other,...,Native,US,US,US,Single,0,0,0,Under Median,1300
19996,TRAIN_19996,20,F,College,Full-Time,12,Education,Admin Support (include Clerical),White,Mexican-American,...,Native,US,Mexico,Mexico,Nonfiler,0,0,0,Under Median,850
19997,TRAIN_19997,22,M,College,Children or Armed Forces,52,Transportation,Technicians & Support,White,All other,...,Native,US,US,US,Single,0,0,0,Unknown,999
19998,TRAIN_19998,76,F,High graduate,Not Working,0,Not in universe or children,Unknown,White,All other,...,Native,US,Scotland,England,Single,0,0,0,Under Median,0


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ID                      20000 non-null  object
 1   Age                     20000 non-null  int64 
 2   Gender                  20000 non-null  object
 3   Education_Status        20000 non-null  object
 4   Employment_Status       20000 non-null  object
 5   Working_Week (Yearly)   20000 non-null  int64 
 6   Industry_Status         20000 non-null  object
 7   Occupation_Status       20000 non-null  object
 8   Race                    20000 non-null  object
 9   Hispanic_Origin         20000 non-null  object
 10  Martial_Status          20000 non-null  object
 11  Household_Status        20000 non-null  object
 12  Household_Summary       20000 non-null  object
 13  Citizenship             20000 non-null  object
 14  Birth_Country           20000 non-null  object
 15  Bi

## train 데이터셋 전처리

In [ ]:
train_df = train_df.drop(columns=['ID'])

In [ ]:
# 범주형 변수 label encoding
label_encoder = LabelEncoder()

categorical_cols = train_df.select_dtypes(include=['object']).columns
for col in categorical_cols:
  train_df[col] = label_encoder.fit_transform(train_df[col].astype(str))

## 훈련 및 검증 데이터 분할

In [ ]:
X = train_df.drop(columns=['Income'])
y = train_df['Income']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## 모델 훈련

In [ ]:
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [ ]:
# 검증 데이터에 대한 예측 및 RMSE 계산
val_preds = model.predict(X_val)
val_rmse = mean_squared_error(y_val, val_preds, squared=False)
print('validation RMSE:', val_rmse)

validation RMSE: 614.2064907327157


## test 데이터셋 전처리

In [ ]:
test_ids = test_df['ID']
test_df = test_df.drop(columns=['ID'])

In [ ]:
for col in categorical_cols:
  test_df[col] = test_df[col].map(lambda x: 'Unknown' if x not in label_encoder.classes_ else x)
  test_df[col] = label_encoder.transform(test_df[col].astype(str))

## test 데이터셋 예측

In [ ]:
test_preds = model.predict(test_df)

## submission 파일 생성

In [ ]:
submission_df = pd.DataFrame({'ID': test_ids, 'Income': test_preds})
submission_df
#submission_df.to_csv(path + 'submission/submission_20240402(1).csv', index=False)